<a href="https://colab.research.google.com/github/nipun-taneja/amorphous-yolo/blob/main/notebooks/01_baseline_vs_eiou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Code

In [ ]:
!pip install --upgrade pip
!pip install -U "ultralytics" "wandb"
!pip install roboflow


In [ ]:
from pathlib import Path

In [ ]:
# Clone your repo into the Colab VM
!git clone https://github.com/nipun-taneja/amorphous-yolo.git
%cd amorphous-yolo
!ls

In [ ]:
import wandb
wandb.login()  # will show a link; paste your API key from wandb.ai

In [ ]:
PROJECT_DIR = "/content/amorphous-yolo"
DRIVE_PROJECT_DIR = "/content/drive/MyDrive/amorphous-yolo"
DATASETS = ["coco8","DUO_dataset","trashcan"]


In [ ]:
%cp -r "/content/drive/MyDrive/amorphous-yolo/datasets" "/content/amorphous-yolo"

## View Ultralytics YOLO Bounding Box Code

In [ ]:
import inspect, ultralytics
from ultralytics.utils import loss as loss_mod

print("Ultralytics version:", ultralytics.__version__)
print(loss_mod.BboxLoss)
print(inspect.getsource(loss_mod.BboxLoss.forward)[:2500])


## Checking if custom loss defined is working

In [ ]:
from src.losses import EIoULoss, AEIoULoss
import torch

print("Ultralytics version:", __import__("ultralytics").__version__)

loss_fn = EIoULoss()
loss_fn_A = AEIoULoss()

pred = torch.tensor([[0.0, 0.0, 1.0, 1.0],
                     [0.2, 0.2, 0.8, 0.8]])
gt   = torch.tensor([[0.0, 0.0, 1.0, 1.0],
                     [0.0, 0.0, 1.0, 1.0]])

print("EIoU placeholder loss:", loss_fn(pred, gt).item())
print("AEIoU placeholder loss:", loss_fn_A(pred, gt).item())

## updating runs in drive

In [ ]:
!mkdir -p "/content/drive/MyDrive/amorphous-yolo"
!cp -r /content/amorphous-yolo/amorphous-yolo/* "/content/drive/MyDrive/amorphous-yolo"

In [ ]:
!mkdir -p "/content/drive/MyDrive/checkpoints/amorphous-yolo"

!cp -r /content/amorphous-yolo/amorphous-yolo/runs/detect/experiments/baseline_your_dataset \
      "/content/drive/MyDrive/checkpoints/amorphous-yolo/"


## YOLOv8 Training Basic (COCO8)

This script demonstrates how to train a YOLO model using the **Ultralytics YOLO API** on the lightweight **COCO8** dataset. It is intended for quick experimentation and validation of the training pipeline.


In [ ]:
from ultralytics import YOLO

model = YOLO("yolo26n.pt")  # works with 8.4.9 as you saw
results = model.train(
    data="coco8.yaml",
    epochs=3,
    imgsz=640,
    project=Path(PROJECT_DIR) /"experiments",
    name="coco_yolo26n_baseline_e3",
    device=0,
)
print(results)


## Getting data from Roboflow.
We already have duo dataset downloaded, no need to re download

In [ ]:
from google.colab import userdata
ROBOFLOW_API_KEY = userdata.get('ROBOFLOW')


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("amorphousyolo").project("duo-dataset-ofte9")
version = project.version(1)
dataset = version.download("yolo26")


## Check if DUO data is correctly accessible

In [ ]:
train_dir = Path(PROJECT_DIR) / "datasets" / DATASETS[1] / "train/images"
label_dir = Path(PROJECT_DIR) / "datasets" / DATASETS[1] / "train/labels"

In [ ]:
print(train_dir)
# /content/amorphous-yolo/datasets/DUO_dataset/train/images

In [ ]:
# This utility function randomly samples images from a dataset and overlays
# their corresponding **YOLO-format bounding boxes** for quick visual inspection
#  useful for sanity-checking labels before training.
import random, cv2, matplotlib.pyplot as plt
from pathlib import Path

def show_random(img_root, lbl_root, n=20):
    img_paths = list(Path(img_root).glob("*.jpg"))
    random.shuffle(img_paths)

    for img_path in img_paths[:n]:
        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]
        label_path = Path(lbl_root) / (img_path.stem + ".txt")

        if label_path.exists():
            with open(label_path) as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue  # skip malformed
                    cls = int(parts[0])
                    x_c, y_c, bw, bh = map(float, parts[1:5])  # ignore extra values

                    x_c *= w; y_c *= h; bw *= w; bh *= h
                    x1, y1 = int(x_c - bw/2), int(y_c - bh/2)
                    x2, y2 = int(x_c + bw/2), int(y_c + bh/2)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        plt.figure(figsize=(4, 4))
        plt.title(img_path.name)
        plt.axis("off")
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()


In [ ]:

imgs = list(train_dir.glob("*"))
print(len(imgs), "files")
print(imgs[:5])

show_random(
    img_root=train_dir,
    lbl_root=label_dir,
    n=5,
)

In [ ]:
# This script is a **low-level debugging utility** for verifying that a specific
#  image and its corresponding YOLO label file exist, are readable, and align
#  correctly when rendered. It is ideal for diagnosing dataset issues such as
# missing labels, malformed annotations, or incorrect paths.
import cv2, matplotlib.pyplot as plt
from pathlib import Path

img_path = train_dir / "4828_jpg.rf.9e0a7e8d443c379bf491e4b866b5a524.jpg"
label_path = label_dir / (img_path.stem + ".txt")

print("Image exists:", img_path.exists())
print("Label path:", label_path)
print("Label exists:", label_path.exists())
if label_path.exists():
    print("Label contents (first 5 lines):")
    with open(label_path) as f:
        for i, line in enumerate(f):
            print(" ", line.strip())
            if i == 4:
                break

img = cv2.imread(str(img_path))
h, w = img.shape[:2]

if label_path.exists():
    with open(label_path) as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            cls = int(parts[0])
            x_c, y_c, bw, bh = map(float, parts[1:5])
            x_c *= w; y_c *= h; bw *= w; bh *= h
            x1, y1 = int(x_c - bw/2), int(y_c - bh/2)
            x2, y2 = int(x_c + bw/2), int(y_c + bh/2)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

plt.figure(figsize=(4,4))
plt.axis("off")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
# This utility visualizes **polygon-based YOLO annotations** (segmentation format)
# by randomly sampling images and drawing a **bounding box derived from polygon
# extents**. It’s useful for validating segmentation labels and quickly
# spotting malformed or mis-scaled polygons.
import random, cv2, matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

def show_random_poly(img_root, lbl_root, n=5):
    img_paths = list(Path(img_root).glob("*.jpg"))
    random.shuffle(img_paths)

    for img_path in img_paths[:n]:
        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]
        label_path = Path(lbl_root) / (img_path.stem + ".txt")
        print("Image:", img_path.name, "Label exists:", label_path.exists())

        if label_path.exists():
            with open(label_path) as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 3:
                        continue
                    cls = int(parts[0])
                    coords = list(map(float, parts[1:]))

                    xs = np.array(coords[0::2]) * w
                    ys = np.array(coords[1::2]) * h

                    x1, x2 = int(xs.min()), int(xs.max())
                    y1, y2 = int(ys.min()), int(ys.max())
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        plt.figure(figsize=(4, 4))
        plt.title(img_path.name)
        plt.axis("off")
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()

show_random_poly(
    img_root= train_dir,
    lbl_root= label_dir,
    n=5,
)


## Processing and getting trashcan data, dont repeat

In [ ]:
!pwd

In [ ]:
!cp /content/drive/MyDrive/amorphous-yolo/data/dataset.zip /content/amorphous-yolo/amorphous-yolo/datasets/trashcan.zip

In [ ]:
!mkdir -p /content/datasets/trashcan_raw

In [ ]:
%cd /content/amorphous-yolo/amorphous-yolo/datasets

In [ ]:
!unzip -q /content/amorphous-yolo/amorphous-yolo/datasets/trashcan.zip

In [ ]:
!ls  /content/amorphous-yolo/amorphous-yolo/datasets/trashcan-raw


In [ ]:
!tree /content/amorphous-yolo/amorphous-yolo/datasets/trashcan-raw

In [ ]:
from pathlib import Path

ROOT = Path("/content/amorphous-yolo/amorphous-yolo/datasets/trashcan-raw")
print("instance_version/train files:", len(list((ROOT/"instance_version/train").glob("*"))))
print("instance_version/val files:", len(list((ROOT/"instance_version/val").glob("*"))))
print("original_data/images files:", len(list((ROOT/"original_data/images").glob("*.jpg"))))


In [ ]:
!pip install pycocotools -q

import os
from pathlib import Path
from pycocotools.coco import COCO

ROOT = Path("/content/amorphous-yolo/amorphous-yolo/datasets/trashcan-raw")
INSTANCE_DIR = ROOT / "instance_version"
IMAGES_DIR = ROOT / "original_data" / "images"

OUT_ROOT = Path("/content/amorphous-yolo/amorphous-yolo/datasets/trashcan")
IMG_TRAIN_DIR = OUT_ROOT / "images" / "train"
IMG_VAL_DIR   = OUT_ROOT / "images" / "val"
LBL_TRAIN_DIR = OUT_ROOT / "labels" / "train"
LBL_VAL_DIR   = OUT_ROOT / "labels" / "val"

for d in [IMG_TRAIN_DIR, IMG_VAL_DIR, LBL_TRAIN_DIR, LBL_VAL_DIR]:
    d.mkdir(parents=True, exist_ok=True)

def coco_to_yolo(json_name, split):
    coco = COCO(str(INSTANCE_DIR / json_name))
    img_id_to_info = coco.imgs
    cat_id_to_idx = {cat_id: i for i, cat_id in enumerate(sorted(coco.cats.keys()))}
    print(f"{split}: {len(img_id_to_info)} images, {len(coco.anns)} anns")

    for img_id, info in img_id_to_info.items():
        file_name = info["file_name"]
        width, height = info["width"], info["height"]

        src_img_path = IMAGES_DIR / file_name
        if split == "train":
            out_img_path = IMG_TRAIN_DIR / file_name
            out_lbl_path = LBL_TRAIN_DIR / (Path(file_name).stem + ".txt")
        else:
            out_img_path = IMG_VAL_DIR / file_name
            out_lbl_path = LBL_VAL_DIR / (Path(file_name).stem + ".txt")

        if not src_img_path.exists():
            continue

        if not out_img_path.exists():
            os.system(f'cp "{src_img_path}" "{out_img_path}"')

        ann_ids = coco.getAnnIds(imgIds=[img_id])
        anns = coco.loadAnns(ann_ids)
        lines = []
        for ann in anns:
            cat_id = ann["category_id"]
            cls = cat_id_to_idx[cat_id]

            x_min, y_min, bw, bh = ann["bbox"]  # COCO bbox
            x_c = (x_min + bw / 2) / width
            y_c = (y_min + bh / 2) / height
            bw_n = bw / width
            bh_n = bh / height

            x_c = min(max(x_c, 0.0), 1.0)
            y_c = min(max(y_c, 0.0), 1.0)
            bw_n = min(max(bw_n, 0.0), 1.0)
            bh_n = min(max(bh_n, 0.0), 1.0)

            lines.append(f"{cls} {x_c:.6f} {y_c:.6f} {bw_n:.6f} {bh_n:.6f}")

        if lines:
            with open(out_lbl_path, "w") as f:
                f.write("\n".join(lines))

coco_to_yolo("instances_train_trashcan.json", "train")
coco_to_yolo("instances_val_trashcan.json", "val")


In [ ]:
from pycocotools.coco import COCO
import yaml
from pathlib import Path

INSTANCE_DIR = Path("/content/amorphous-yolo/amorphous-yolo/datasets/trashcan-raw/instance_version")
OUT_ROOT = Path("/content/amorphous-yolo/amorphous-yolo/datasets/trashcan")

coco_train = COCO(str(INSTANCE_DIR / "instances_train_trashcan.json"))
cats = coco_train.loadCats(coco_train.getCatIds())
cats_sorted = sorted(cats, key=lambda c: c["id"])
names = {i: c["name"] for i, c in enumerate(cats_sorted)}
print("names:", names)

data_dir = Path("/content/amorphous-yolo/amorphous-yolo/data")
data_dir.mkdir(parents=True, exist_ok=True)

yaml_dict = {
    "path": str(OUT_ROOT),
    "train": "images/train",
    "val": "images/val",
    "names": names,
}

with open(data_dir / "trashcan.yaml", "w") as f:
    yaml.safe_dump(yaml_dict, f, sort_keys=False)

print((data_dir / "trashcan.yaml").read_text())


In [ ]:
import random, cv2, matplotlib.pyplot as plt
from pathlib import Path

def show_random_trashcan(img_root, lbl_root, n=10):
    img_paths = list(Path(img_root).glob("*.jpg"))
    random.shuffle(img_paths)

    for img_path in img_paths[:n]:
        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]
        label_path = Path(lbl_root) / (img_path.stem + ".txt")
        print("Image:", img_path.name, "Label exists:", label_path.exists())

        if label_path.exists():
            for line in open(label_path):
                parts = line.strip().split()
                if len(parts) < 5:
                    continue
                _, x_c, y_c, bw, bh = map(float, parts[:5])
                x_c *= w; y_c *= h; bw *= w; bh *= h
                x1, y1 = int(x_c - bw/2), int(y_c - bh/2)
                x2, y2 = int(x_c + bw/2), int(y_c + bh/2)
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        plt.figure(figsize=(4,4))
        plt.axis("off")
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()

show_random_trashcan(
    "/content/amorphous-yolo/amorphous-yolo/datasets/trashcan/images/train",
    "/content/amorphous-yolo/amorphous-yolo/datasets/trashcan/labels/train",
    n=10,
)


# “Phase 2 – DUO baselines”

 # DUO-CIOU baseline

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo26n.pt")
results = model.train(
    data=Path(PROJECT_DIR) / "data" / "duo.yaml",   # DUO Dataset yaml
    epochs=20,
    imgsz=640,
    project=Path(PROJECT_DIR) /"experiments",
    name="duo_yolo26n_baseline_e20",
    device=0,
)


## Patching YOLO Bounding Box Loss to Use EIoU

This script **monkey-patches Ultralytics YOLO’s internal bounding box loss** to replace the default **CIoU-based IoU loss** with a custom **EIoU (Extended IoU) loss**, while leaving the rest of the loss pipeline unchanged.

This allows experimentation with alternative IoU formulations **without forking or modifying Ultralytics source code**.

---

In [ ]:
import torch
import ultralytics
from ultralytics.utils import loss as loss_mod
from src.losses import EIoULoss  # the file you showed

print("Ultralytics version:", ultralytics.__version__)

# Keep a handle to the original if you ever want to restore it
OriginalBboxLossForward = loss_mod.BboxLoss.forward


def bbox_loss_forward_eiou(
    self,
    pred_dist: torch.Tensor,
    pred_bboxes: torch.Tensor,
    anchor_points: torch.Tensor,
    target_bboxes: torch.Tensor,
    target_scores: torch.Tensor,
    target_scores_sum: torch.Tensor,
    fg_mask: torch.Tensor,
    imgsz: torch.Tensor,
    stride: torch.Tensor,
):
    # Same weighting as original CIoU branch
    weight = target_scores.sum(-1)[fg_mask].unsqueeze(-1)

    # EIoU per-box loss in xyxy
    eiou_loss_fn = EIoULoss(reduction="none")
    eiou_loss = eiou_loss_fn(pred_bboxes[fg_mask], target_bboxes[fg_mask])  # shape [N]

    # Match original pattern: (loss * weight).sum() / target_scores_sum
    loss_iou = (eiou_loss.unsqueeze(-1) * weight).sum() / target_scores_sum

    # DFL branch (unchanged from original BboxLoss.forward)
    if self.dfl_loss:
        target_ltrb = loss_mod.bbox2dist(anchor_points, target_bboxes, self.dfl_loss.reg_max - 1)
        loss_dfl = self.dfl_loss(
            pred_dist[fg_mask].view(-1, self.dfl_loss.reg_max),
            target_ltrb[fg_mask],
        ) * weight
        loss_dfl = loss_dfl.sum() / target_scores_sum
    else:
        target_ltrb = loss_mod.bbox2dist(anchor_points, target_bboxes)
        # normalize ltrb by image size
        target_ltrb = target_ltrb * stride
        target_ltrb[..., 0::2] /= imgsz[1]
        target_ltrb[..., 1::2] /= imgsz[0]
        pred_dist = pred_dist * stride
        pred_dist[..., 0::2] /= imgsz[1]
        pred_dist[..., 1::2] /= imgsz[0]
        loss_dfl = (
            torch.nn.functional.l1_loss(
                pred_dist[fg_mask],
                target_ltrb[fg_mask],
                reduction="none",
            ).mean(-1, keepdim=True)
            * weight
        )
        loss_dfl = loss_dfl.sum() / target_scores_sum

    return loss_iou, loss_dfl


# Apply the patch
loss_mod.BboxLoss.forward = bbox_loss_forward_eiou
print("Patched BboxLoss.forward to use EIoU.")


In [ ]:
from ultralytics import YOLO

# Load the same model as your CIoU baseline
model = YOLO("yolo26n.pt")

results = model.train(
    data=Path(PROJECT_DIR) / "data" / "duo.yaml",   # same DUO yaml as baseline
    epochs=20,
    imgsz=640,
    project=Path(PROJECT_DIR) /"experiments",
    name="duo_yolo26_eiou_e20",      # new run name
    device=0,
)


In [ ]:
import inspect
from ultralytics.utils import loss as loss_mod

print(inspect.getsource(loss_mod.BboxLoss.forward)[:2400])


In [ ]:
import torch
import ultralytics
from ultralytics.utils import loss as loss_mod
from src.losses_new import AEIoULoss  # new class above

print("Ultralytics version:", ultralytics.__version__)

# Choose λ-rigidity here
RIGIDITY = 1.0  # later change to 0.1 for A-EIoU "amorphous mode"

OriginalBboxLossForward = loss_mod.BboxLoss.forward


def bbox_loss_forward_aeiou(
    self,
    pred_dist: torch.Tensor,
    pred_bboxes: torch.Tensor,
    anchor_points: torch.Tensor,
    target_bboxes: torch.Tensor,
    target_scores: torch.Tensor,
    target_scores_sum: torch.Tensor,
    fg_mask: torch.Tensor,
    imgsz: torch.Tensor,
    stride: torch.Tensor,
):
    """Compute A-EIoU and DFL losses for bounding boxes."""
    weight = target_scores.sum(-1)[fg_mask].unsqueeze(-1)

    # A-EIoU per-box loss in xyxy
    aeiou_loss_fn = AEIoULoss(rigidity=RIGIDITY, reduction="none")
    aeiou_loss = aeiou_loss_fn(pred_bboxes[fg_mask], target_bboxes[fg_mask])  # [N]

    loss_iou = (aeiou_loss.unsqueeze(-1) * weight).sum() / target_scores_sum

    # DFL branch unchanged from original
    if self.dfl_loss:
        target_ltrb = loss_mod.bbox2dist(anchor_points, target_bboxes, self.dfl_loss.reg_max - 1)
        loss_dfl = self.dfl_loss(
            pred_dist[fg_mask].view(-1, self.dfl_loss.reg_max),
            target_ltrb[fg_mask],
        ) * weight
        loss_dfl = loss_dfl.sum() / target_scores_sum
    else:
        target_ltrb = loss_mod.bbox2dist(anchor_points, target_bboxes)
        target_ltrb = target_ltrb * stride
        target_ltrb[..., 0::2] /= imgsz[1]
        target_ltrb[..., 1::2] /= imgsz[0]
        pred_dist = pred_dist * stride
        pred_dist[..., 0::2] /= imgsz[1]
        pred_dist[..., 1::2] /= imgsz[0]
        loss_dfl = (
            torch.nn.functional.l1_loss(
                pred_dist[fg_mask],
                target_ltrb[fg_mask],
                reduction="none",
            ).mean(-1, keepdim=True)
            * weight
        )
        loss_dfl = loss_dfl.sum() / target_scores_sum

    return loss_iou, loss_dfl


loss_mod.BboxLoss.forward = bbox_loss_forward_aeiou
print(f"Patched BboxLoss.forward to use A-EIoU with rigidity={RIGIDITY}.")


In [ ]:
import torch
from src.losses_new import AEIoULoss

print(AEIoULoss.__init__)
help(AEIoULoss)


In [ ]:
from ultralytics import YOLO

model = YOLO("yolo26n.pt")
results = model.train(
    data=Path(PROJECT_DIR) / "data" / "duo.yaml",
    epochs=20,
    imgsz=640,
    project=Path(PROJECT_DIR) /"experiments",
    name="duo_yolo26_aeiou_r1_e20",
    device=0,
)


In [ ]:
RIGIDITY = 0.1


In [ ]:
model = YOLO("yolo26n.pt")
results = model.train(
    data=dataset.location + "/data.yaml",
    epochs=20,
    imgsz=640,
    project="experiments",
    name="duo_yolo26_aeiou_r0p1_e20",
    device=0,
)
